In [6]:
# !pip install mxnet
# !pip install gluonnlp
# !pip install sentencepiece
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
# !pip install transformers==3.0.2

## Library import

In [1]:
import os
import random
import numpy as np
import pandas as pd
import gluonnlp as nlp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, tqdm_notebook
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, load_metric, load_from_disk
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding, EarlyStoppingCallback, AdamW

from transformers.optimization import get_cosine_schedule_with_warmup
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

### Device & Random_seed setting

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore

seed_everything(42)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


---

### Kobert for Hugging Face

In [3]:
MODEL = 'skt/kobert-base-v1'
model = AutoModel.from_pretrained(MODEL)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.truncation_side = 'left'

In [4]:
# print(model)

In [5]:
MAX_LEN = 512
_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def tokenized_fn(data):
    outputs = tokenizer(data["article"], padding=True, max_length=MAX_LEN, truncation=True)
    if 'label' in data:
        outputs["labels"] = data["label"]
    return outputs

def metric_fn(p):
    preds, labels = p
    output = _metric.compute(references=lables, predictions=np.argmax(preds, axis=-1))
    return output


train_dataset = load_dataset("csv", data_files="./data/train_report.csv")["train"]
valid_dataset = load_dataset("csv", data_files="./data/valid_report.csv")["train"]

train_dataset = train_dataset.map(tokenized_fn, remove_columns=['filename', 'article', 'length', 'label'])
valid_dataset = valid_dataset.map(tokenized_fn, remove_columns=['filename', 'article', 'length', 'label'])

Using custom data configuration default-e8f1ba7701d4e902
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-e8f1ba7701d4e902/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-21a9e1aedcee44a1
Reusing dataset csv (/home/piai/.cache/huggingface/datasets/csv/default-21a9e1aedcee44a1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/piai/.cache/huggingface/datasets/csv/default-e8f1ba7701d4e902/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-f24d4418ea6a3ed7.arrow
Loading cached processed dataset at /home/piai/.cache/huggingface/datasets/csv/default-21a9e1aedcee44a1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-37a56cd247a4152b.arrow


In [6]:
class ClassificationHead(nn.Module):
    def __init__(self):
        super().__init__()
        # self.dense1 = torch.nn.Linear(768, 3072)
        self.dropout = torch.nn.Dropout(0.25)
        # self.dense2 = torch.nn.Linear(3072, 768)
        self.output = torch.nn.Linear(768, 2)
    
    def forward(self, features):
        # 보통 분류기에선 start 토큰에 분류 결과를 담음
        x = features[:, 0, :]    # take <s> token (equiv. to [CLS])
        x = x.reshape(-1, x.size(-1))
        x = self.dropout(x)
        
#         x = self.dense1(x)
#         x = torch.relu(x)
#         x = self.dropout(x)
        
#         x = self.dense2(x)
#         x = torch.tanh(x)
#         x = self.dropout(x)
        x = self.output(x)
        return x

class KobertForAinalyst(nn.Module):
    def __init__(self, model):
        super(KobertForAinalyst, self).__init__()
        self.model = model
        self.classifier = ClassificationHead()
    
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # labels=labels
        )
        self.labels = labels
        logits = self.classifier(outputs["last_hidden_state"])
        # prob = torch.nn.functional.softmax(logits, dim=-1)
        
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return logits, loss
        else:
            return logits

In [7]:
# model = nn.DataParallel(model)
# isParallel = True

In [8]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [9]:
batch_size = 16

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    sampler = torch.utils.data.RandomSampler(train_dataset),
    batch_size = batch_size,
    collate_fn = _collator,
)

valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    sampler = torch.utils.data.SequentialSampler(valid_dataset),
    batch_size = batch_size,
    collate_fn = _collator,
)

# args = TrainingArguments(
#     'runs/',
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=16,
#     learning_rate=2e-5,
#     num_train_epochs=30,
#     save_total_limit=5,
#     do_train=True,
#     do_eval=True,
#     # eval_epochs=1,
#     save_strategy="epoch",
#     logging_strategy="epoch",
#     evaluation_strategy="epoch",
#     load_best_model_at_end = True,
#     # metric_for_best_model = 'f1',
# )

# trainer = Trainer(
#         model=model,
#         args=args,
#         data_collator=_collator,
#         train_dataset=train_dataset,
#         eval_dataset=valid_dataset,
#         tokenizer=tokenizer,
#         compute_metrics=metric_fn,
#         callbacks=[EarlyStoppingCallback(early_stopping_patience = 5)]
# )

# trainer.train()

In [10]:
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [12]:
epochs = 30
for epoch in range(epochs):
    print(f"==================== Epoch {epoch+1}/{epochs} ====================")
    print("Training...")
    
    total_train_loss = 0
    model.train()
    
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
        
        model.zero_grad()
        
        logits, loss = model(
            input_ids = batch_input_ids,
            attention_mask = batch_attention_mask,
            labels = batch_labels,
        )
        
        # # output 모양 출력해보기
        # outputs = model(
        #     input_ids = batch_input_ids,
        #     attention_mask = batch_attention_mask,
        #     labels = batch_labels,
        # )
        # print("print outputs : ")
        # print(outputs)
        # print(outputs.last_hidden_state.shape)
        # break
    
        if isParallel:
            loss = loss.mean()

        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
        if step % 1000 == 0 and not step == 0:
            print("step : {:>5,} of {:>5,} loss: {:.5f}".format(step, len(train_dataloader), loss.item()))
    
    avg_train_loss = total_train_loss / len(train_dataloader)
    print("    >>> Average training loss: {0:.5f}".format(avg_train_loss))
    
    # Validation
    print()
    print("Running Validation...")
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch_input_ids = batch["input_ids"].to(device)
        batch_attention_mask = batch["attention_mask"].to(device)
        batch_labels = batch["labels"].to(device)
    
        with torch.no_grad():
            logits, loss = model(
                input_ids = batch_input_ids,
                attention_mask = batch_attention_mask,
                labels = batch_labels,
            )
            
            if isParallel:
                loss = loss.mean()
            
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = batch_labels.to("cpu").numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
    print("    >>> Accuracy: {0:.5f}".format(avg_val_accuracy))

==================== Epoch 1/30 ====================
Training...


  0%|                                                                                                                         | 0/138 [00:00<?, ?it/s]


TypeError: forward() got an unexpected keyword argument 'labels'

---
#### Kobert by using kobert library

In [7]:
model, vocab = get_pytorch_kobert_model()

/home/piai/hustar/Hustar_Group_4_TeamP/testMH/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/home/piai/hustar/Hustar_Group_4_TeamP/testMH/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [8]:
train_dataset = nlp.data.TSVDataset('./data/train_report.tsv', field_indices=[1,3], num_discard_samples=1)
valid_dataset = nlp.data.TSVDataset('./data/valid_report.tsv', field_indices=[1,3], num_discard_samples=1)

In [9]:
train_dataset[0]

['하지만 이번 경기 Cycle 에는 이자유예, 양극화 등 여러 요인으로 자산건전성이 경기에 후행할 수도 있다. 2021 년 순이익 추정치를 1 조 5,560 억원으로 6% 상향조정하지만, 여느 때에 비해 실적의 불확실성은 높은 편이다. 장기적인 배당 불확실성 감안하여 목표주가 9,000 원으로 하향조정기업은행은 국책은행이어서 금감원의 배당 축소 권고 대상에 포함되지 않는다. 그럼에도 불구하고 과연 배당성향을 더 높일 수 있을지는 불확실하다. 앞으로도 정책적인 역할을 중시할 가능성이 있기 때문이다. 장기 배당성향 기대치를 35%에서 30%로 낮추면서목표주가를 10,000 원에서 9,000 원으로 하향조정한다. 기존의 ‘중립’ 의견을 유지한다.',
 '0']

In [10]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
max_len = 512
batch_size = 16
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate = 2e-5

In [14]:
# #토큰화
# tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)

# #BERTDataset 클래스 이용, TensorDataset으로 만들어주기
# train_dataset = BERTDataset(train_dataset, 0, 1, tokenizer, max_len, True, False)
# valid_dataset = BERTDataset(valid_dataset, 0, 1, tokenizer, max_len, True, False)